In [ ]:
import pandas as pd
import altair as alt
import numpy as np

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)
pd.set_option('display.max_columns', None)
alt.data_transformers.disable_max_rows()

In [ ]:
df = pd.read_csv("../data/raw/SpeedDating.csv",delimiter=",")

In [ ]:
df.head()

In [ ]:
df = df.dropna(subset=["like", "attr4_3"])

In [ ]:
df["like"]

In [ ]:
# alt.Chart(df).mark_bar().encode(
#     alt.X("you_call:O"),
#     alt.Y("count(you_call)")
# ) | alt.Chart(df).mark_bar().encode(
#     alt.X("them_cal:O"),
#     alt.Y("count(them_cal)")
# )


In [ ]:
# c1 = alt.Chart(df).mark_bar().encode(
#     alt.X("samerace:O"),
#     alt.Y("mean(imprace)")
# )

# c2 = alt.Chart(df).mark_bar().encode(
#     alt.X("samerace:O"),
#     alt.Y("mean(imprelig)")
# )

# c1 | c2

In [ ]:
# alt.Chart(df).mark_bar().encode(
#     alt.X("age_o"),
#     alt.Y("count(match)"),
#     alt.Color("match:O")
# )

In [ ]:
# alt.Chart(df).mark_bar().encode(
#     alt.X("condtn:O", axis = alt.Axis(title="Conditional")),
#     alt.Y("count(match)"),
#     alt.Color("match:O")
# )

In [ ]:
y_label = "like"

X = df[["attr4_3"]]
y = df[y_label]

In [ ]:
alt.Chart(df).mark_circle().encode(
    x='attr4_3',
    y='like'
)

## Use Pearson correlation matrix

In [ ]:
corr = df.corr(method='pearson').round(2)
corr_blues = corr.style.background_gradient(cmap='Blues')

In [ ]:
corr_blues

In [ ]:
corr_list = corr[y_label].sort_values(ascending=False)

In [ ]:
corr_list

## Linear regression (numerical)

In [ ]:
# Choose the linear regression model
reg = LinearRegression()

In [ ]:
reg.fit(X, y)

In [ ]:
print(f' Intercept: {reg.intercept_} \n Slope: {reg.coef_[0]}')

In [ ]:
y_pred = reg.predict(X)

In [ ]:
mean_squared_error(y, y_pred)

In [ ]:
mean_squared_error(y, y_pred, squared=False)

In [ ]:
# add prediction to DF
df['y_pred'] = y_pred

In [ ]:
chart = alt.Chart(df).mark_point().encode(
    x='attr4_3',
    y=y_label
)

chart + chart.transform_regression('attr4_3', y_label).mark_line()

In [ ]:
reg.score(X, y)

## Linear regression categorical

In [ ]:
df_new = df.copy()

This will convert every categorical variable in a new column with a numerical value:

In [ ]:
df_new = pd.get_dummies(df_new)

In [ ]:
df_new

In [ ]:
df_new = df_new.dropna(subset=["like", "amb4_2","fun4_2","intel4_2","sinc4_2","attr4_2", "shar4_2"])
list_numeric = ["amb4_2","fun4_2","intel4_2","sinc4_2","attr4_2", "shar4_2"]

X_new = df_new[["amb4_2","fun4_2","intel4_2","sinc4_2","attr4_2", "shar4_2"]]
y_new = df_new[["like"]]

In [ ]:
y_new = df_new["like"]
y_new

In [ ]:
alt.Chart(df_new).mark_circle().encode(
    x=alt.X(alt.repeat("column"), 
            type='quantitative',
            scale=alt.Scale(zero=False)
            ),
    y=alt.Y(alt.repeat("row"), 
            type='quantitative',
            scale=alt.Scale(zero=False)
             ),
    color=alt.Color("like", scale=alt.Scale(range=['blue','green', 'red']))
).properties(
    width=150,
    height=150
).repeat(
    row=list_numeric,
    column=list_numeric
)

In [ ]:
reg.fit(X_new, y_new)

In [ ]:
y_pred = reg.predict(X_new)

In [ ]:
r2_score(y, y_pred)

In [ ]:
mean_squared_error(y, y_pred, squared=False)

In [ ]:
df_new['y_pred'] = y_pred

In [ ]:
alt.Chart(df_new).mark_bar().encode(
    x='field_Mathematics',
    y='sum(y_pred)'
)